In [1]:
from AttentionModule import Conv2d_Attn

import torch
from torch import nn
from torchvision import models, datasets, transforms
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

import re
import numpy as np

In [2]:
resnet_pretrained = models.resnet50(pretrained=True)
nn.Conv2d = Conv2d_Attn
resnet_attn = models.resnet50()
resnet_attn.load_state_dict(resnet_pretrained.state_dict(), strict=False)

In [3]:
# This block turns 'layer1.0.downsample.0.weight' to 'layer1[0].downsample[0].weight'
param_keys = list(resnet_attn.state_dict().keys())
formatted_keys = []
for k in param_keys:
    found = re.findall(r'\.[\d]{1,2}\.', k)
    if len(found):
        for f in found:
            k = k.replace(f, '[{}].'.format(f.strip('.')))
    formatted_keys.append(k)

In [4]:
# This block turn off gradient up for all params except attn_weights
def turn_off_grad_except(lst=[]):
    turning_off_keys = [k for k in formatted_keys for l in lst if l in k]
    for k in formatted_keys:
        obj = eval('resnet_attn.'+k)
        if k in turning_off_keys:
            obj.requires_grad = True
        else:
            obj.requires_grad = False

In [5]:
resnet_attn.fc = nn.Linear(resnet_attn.fc.in_features, 144)

Start training

In [6]:
batch_size = 32

In [7]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(),
     normalize])

trainset = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [8]:
total_imgs = len(trainset.imgs)

In [9]:
resnet_attn = resnet_attn.cuda()

In [10]:
total_attn_params = 0
for k in formatted_keys:
    obj = eval('resnet_attn.'+k)
    if 'attn_weights' in k:
        total_attn_params += np.prod(obj.shape)
print("Total number of attention parameters", total_attn_params)

Total number of attention parameters 26560


We want the attention parameters to diverge from 1, therefore we penalize element-wise square loss as $\lambda (1 \times \text{# params} - (x - 1)^2)$

But this is too big a number,
let's try: 
$- (x - 1)^2$ for now

In [11]:
_lambda = 1e-1 #set default

In [12]:
def get_params_objs(name, net='resnet_attn'):
    res = []
    for k in formatted_keys:
        obj = eval(f'{net}.'+k)
        if name in k:
            res.append(obj)
    return res

In [13]:
def compute_attn_loss(n_params=26560):
    attns = get_params_objs('attn_weights')
#     penality = sum([torch.pow(t - 1,2).mean() for t in attns])
    penality = (1000/n_params)*sum([torch.min(torch.pow(t-2, 2), torch.pow(t, 2)).sum() for t in attns])
    return (_lambda)*(penality)

In [14]:
len(get_params_objs('attn_weights'))

53

In [15]:
compute_attn_loss()

Variable containing:
 100.0000
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [16]:
print_every = 5

In [17]:
def train_k_epoch(k, add_attn=True, score_epoch=False):
    cls_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet_attn.parameters()))
    
    for epoch in range(k):
        running_loss = 0.0
        running_attn_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()

            optimizer.zero_grad()
            outputs = resnet_attn(inputs)
            clf_loss = cls_criterion(outputs, labels)
                        
            attn_loss = compute_attn_loss()
            
            if add_attn:
                loss = clf_loss + attn_loss
            else:
                loss = clf_loss

            loss.backward()
            optimizer.step()

            running_loss += clf_loss.data[0]
            running_attn_loss += attn_loss.data[0]

            if i % print_every == 0:
                print('[%5d] iter, [%2f] epoch, classifer loss: %.3f, attn_loss: %.5f ' %
                      (i + 1, i*batch_size/total_imgs, running_loss/print_every, running_attn_loss/print_every))
                running_loss = 0.0
                running_attn_loss = 0.0
        if score_epoch:
            print(score(batch_size=32))

In [18]:
# def train_one_epoch(add_attn=True):
#     cls_criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet_attn.parameters()))
    
#     running_loss = 0.0
#     running_attn_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         inputs, labels = data
#         inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()

#         optimizer.zero_grad()
#         outputs = resnet_attn(inputs)
#         clf_loss = cls_criterion(outputs, labels)
#         attn_loss = compute_attn_loss()
#         if add_attn:
#             loss = clf_loss + attn_loss
#         else:
#             loss = clf_loss

#         loss.backward()
#         optimizer.step()

#         running_loss += clf_loss.data[0]
#         running_attn_loss += attn_loss.data[0]

#         if i % print_every == 0:
#             print('[%5d] iter, [%2f] epoch, classifer loss: %.3f, attn_loss: %.5f ' %
#                   (i + 1, i*batch_size/total_imgs, running_loss/print_every, running_attn_loss/print_every))
#             running_loss = 0.0
#             running_attn_loss = 0.0

In [19]:
from tqdm import tqdm
def score(net=resnet_attn, batch_size=batch_size):
    trainset = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    
    valset = torchvision.datasets.ImageFolder(root='./data/val', transform=transform)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    
    train_correct = 0
    val_correct = 0
    
    for inp, label in tqdm(iter(trainloader)):
        _, idx = net(Variable(inp).cuda()).max(1)
        train_correct += int(sum(idx.cpu().data == label))
    
    for inp, label in tqdm(iter(valloader)):
        _, idx = net(Variable(inp).cuda()).max(1)
        val_correct += int(sum(idx.cpu().data == label))
    
    return {
        'train_accu': train_correct/len(trainset),
        'val_accu': val_correct/len(valset)
    }

Train a fresh fc layer. 
`turn_off_grad_except([])` turns off grads for all weights but the fc layer

In [20]:
turn_off_grad_except(['attn_weights','fc'])
resnet_attn.eval()
train_k_epoch(5,score_epoch=True)

[    1] iter, [0.000000] epoch, classifer loss: 0.996, attn_loss: 20.00000 
[    6] iter, [0.019605] epoch, classifer loss: 4.831, attn_loss: 99.40570 
[   11] iter, [0.039211] epoch, classifer loss: 4.762, attn_loss: 98.43576 
[   16] iter, [0.058816] epoch, classifer loss: 4.679, attn_loss: 97.47825 
[   21] iter, [0.078422] epoch, classifer loss: 4.451, attn_loss: 96.52525 
[   26] iter, [0.098027] epoch, classifer loss: 4.258, attn_loss: 95.57863 
[   31] iter, [0.117633] epoch, classifer loss: 4.257, attn_loss: 94.63789 
[   36] iter, [0.137238] epoch, classifer loss: 3.738, attn_loss: 93.70402 
[   41] iter, [0.156844] epoch, classifer loss: 4.058, attn_loss: 92.77974 
[   46] iter, [0.176449] epoch, classifer loss: 3.857, attn_loss: 91.86414 
[   51] iter, [0.196054] epoch, classifer loss: 3.884, attn_loss: 90.95941 
[   56] iter, [0.215660] epoch, classifer loss: 3.618, attn_loss: 90.06505 
[   61] iter, [0.235265] epoch, classifer loss: 3.912, attn_loss: 89.17942 
[   66] iter

100%|██████████| 55/55 [00:13<00:00,  4.02it/s]

{'train_accu': 0.24200465629212106, 'val_accu': 0.2394526795895097}


[    1] iter, [0.000000] epoch, classifer loss: 0.664, attn_loss: 11.96036 
[    6] iter, [0.019605] epoch, classifer loss: 3.123, attn_loss: 59.43118 
[   11] iter, [0.039211] epoch, classifer loss: 3.262, attn_loss: 58.82069 
[   16] iter, [0.058816] epoch, classifer loss: 3.433, attn_loss: 58.21531 
[   21] iter, [0.078422] epoch, classifer loss: 3.313, attn_loss: 57.61526 
[   26] iter, [0.098027] epoch, classifer loss: 3.285, attn_loss: 57.02174 
[   31] iter, [0.117633] epoch, classifer loss: 3.444, attn_loss: 56.43397 
[   36] iter, [0.137238] epoch, classifer loss: 3.386, attn_loss: 55.84815 
[   41] iter, [0.156844] epoch, classifer loss: 3.427, attn_loss: 55.26619 
[   46] iter, [0.176449] epoch, classifer loss: 3.463, attn_loss: 54.68930 
[   51] iter, [0.196054] epoch, classifer loss: 3.310, attn_loss: 54.11746 
[   56] iter, [0.215660] epoch, classifer loss: 3.388, attn_loss: 53.55119 
[   61] iter, [0.235265] epoch, classifer loss: 3.397, attn_loss: 52.98815 
[   66] iter

100%|██████████| 55/55 [00:13<00:00,  4.00it/s]

{'train_accu': 0.16591104031368706, 'val_accu': 0.1619156214367161}


[    1] iter, [0.000000] epoch, classifer loss: 0.731, attn_loss: 6.82167 
[    6] iter, [0.019605] epoch, classifer loss: 3.771, attn_loss: 33.86945 
[   11] iter, [0.039211] epoch, classifer loss: 3.663, attn_loss: 33.47488 
[   16] iter, [0.058816] epoch, classifer loss: 3.886, attn_loss: 33.08380 
[   21] iter, [0.078422] epoch, classifer loss: 3.682, attn_loss: 32.69592 
[   26] iter, [0.098027] epoch, classifer loss: 3.740, attn_loss: 32.31085 
[   31] iter, [0.117633] epoch, classifer loss: 3.768, attn_loss: 31.92910 
[   36] iter, [0.137238] epoch, classifer loss: 3.587, attn_loss: 31.55175 
[   41] iter, [0.156844] epoch, classifer loss: 3.697, attn_loss: 31.17883 
[   46] iter, [0.176449] epoch, classifer loss: 3.866, attn_loss: 30.80954 
[   51] iter, [0.196054] epoch, classifer loss: 3.682, attn_loss: 30.44321 
[   56] iter, [0.215660] epoch, classifer loss: 3.725, attn_loss: 30.08019 
[   61] iter, [0.235265] epoch, classifer loss: 3.801, attn_loss: 29.72103 
[   66] iter,

 14%|█▍        | 36/256 [00:08<00:54,  4.00it/s]Process Process-15:
Process Process-16:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/ubuntu/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/ubuntu/miniconda3/lib/python3.6/multip

RuntimeError: DataLoader worker (pid 8975) exited unexpectedly with exit code 1.

In [27]:
compute_attn_loss()

Variable containing:
 1.8107
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [26]:
_lambda = 0.01

In [ ]:
turn_off_grad_except(['attn_weights','fc'])
resnet_attn.eval()
train_k_epoch(5,score_epoch=True)

[    1] iter, [0.000000] epoch, classifer loss: 0.739, attn_loss: 0.36215 
[    6] iter, [0.019605] epoch, classifer loss: 4.000, attn_loss: 1.79801 
[   11] iter, [0.039211] epoch, classifer loss: 4.009, attn_loss: 1.77731 
[   16] iter, [0.058816] epoch, classifer loss: 4.231, attn_loss: 1.75720 
[   21] iter, [0.078422] epoch, classifer loss: 4.062, attn_loss: 1.73770 
[   26] iter, [0.098027] epoch, classifer loss: 4.408, attn_loss: 1.71858 
[   31] iter, [0.117633] epoch, classifer loss: 4.088, attn_loss: 1.69986 
[   36] iter, [0.137238] epoch, classifer loss: 4.138, attn_loss: 1.68110 
[   41] iter, [0.156844] epoch, classifer loss: 4.120, attn_loss: 1.66234 
[   46] iter, [0.176449] epoch, classifer loss: 3.861, attn_loss: 1.64360 
[   51] iter, [0.196054] epoch, classifer loss: 3.969, attn_loss: 1.62500 
[   56] iter, [0.215660] epoch, classifer loss: 3.999, attn_loss: 1.60674 
[   61] iter, [0.235265] epoch, classifer loss: 3.966, attn_loss: 1.58895 
[   66] iter, [0.254871] 

100%|██████████| 55/55 [00:13<00:00,  4.02it/s]

{'train_accu': 0.12804803332924886, 'val_accu': 0.11573546180159636}


[    1] iter, [0.000000] epoch, classifer loss: 0.766, attn_loss: 0.22158 
[    6] iter, [0.019605] epoch, classifer loss: 3.891, attn_loss: 1.10353 
[   11] iter, [0.039211] epoch, classifer loss: 4.020, attn_loss: 1.09631 
[   16] iter, [0.058816] epoch, classifer loss: 3.856, attn_loss: 1.08919 
[   21] iter, [0.078422] epoch, classifer loss: 3.748, attn_loss: 1.08223 
[   26] iter, [0.098027] epoch, classifer loss: 3.714, attn_loss: 1.07529 
[   31] iter, [0.117633] epoch, classifer loss: 3.795, attn_loss: 1.06859 
[   36] iter, [0.137238] epoch, classifer loss: 3.809, attn_loss: 1.06220 
[   41] iter, [0.156844] epoch, classifer loss: 4.083, attn_loss: 1.05602 
[   46] iter, [0.176449] epoch, classifer loss: 4.025, attn_loss: 1.05003 
[   51] iter, [0.196054] epoch, classifer loss: 3.865, attn_loss: 1.04402 
[   56] iter, [0.215660] epoch, classifer loss: 3.657, attn_loss: 1.03789 
[   61] iter, [0.235265] epoch, classifer loss: 3.623, attn_loss: 1.03174 
[   66] iter, [0.254871] 